# Analyse du jeu de données

Dans un premier temps, nous allons explorer le jeu données mis à notre disposition. L'objectif est de repérer les éventuelles anomalies comme par exemple:

 * des données abérantes (ex : dimension);
 * des données manquantes, auquel cas nous proposerons une stratégies pour palier ce problème;
 * la non conformité avec l'énoncé.
 
Pour effectuer cette phase d'analyse exploratoire, nous allons utiliser la bibliothèque **pandas**.

In [1]:
import pandas as pd 

In [2]:
# Chemin vers le fichier csv contenant les données à explorer.
path = "data/original/mower_market_snapshot.csv"

# Délimiteur utilisé dans le fichier csv
delimiter=";"

# Création de la dataframe à partir du fichier csv
mower_df = pd.read_csv(path, delimiter=delimiter)

## 1. Déscription générale de la dataframe

Nous allons utiliser la méthode **describe** pour obtenir une première synthèse globale des données

Pour les variables continues, les information affichées par cette méthode sont : 

* count qui permet de voir le nombre de cellules non vides par colonnes;
* mean, la moyenne par colonnes;
* std, la déviation standard par colonnes;
* min, la valeur miniale par colonnes;
* 25%, la valeur du 25ème centile par colonnes;
* 50%, la médiane par colonnes;
* 75%, la valeur du 75ème centile par colonnes;
* max, la valeur maximale par colonnes.

Pour les variables catégorielles, l'information intéréssante est la valeur **unique** qui permet de calculer le nombre de varibles distinctes par colonnes.

In [3]:
mower_df.describe(include='all')

,capacity,failure_rate,id,margin,price,prod_cost,product_type,quality,warranty,market_share,attractiveness
count,1399.000000,1399.000000,1399.000000,1399.000000,1399.000000,1381,1399,1399,1399,1399.000000,1399.000000
unique,NaN,NaN,NaN,NaN,NaN,1333,3,3,27,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,0,essence,Low,1 an.,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,20,844,971,103,NaN,NaN
mean,47.663266,0.166448,18958.000000,186.262609,260.075165,NaN,NaN,NaN,NaN,0.002144,0.636507
std,19.840501,0.073605,404.000825,191.011102,259.949127,NaN,NaN,NaN,NaN,0.001790,0.124016
min,0.013884,0.028906,18259.000000,52.943989,62.162200,NaN,NaN,NaN,NaN,0.000564,0.297384
25%,35.853890,0.095678,18608.500000,62.247602,73.379425,NaN,NaN,NaN,NaN,0.001135,0.549369
50%,50.309242,0.199796,18958.000000,66.299672,78.439753,NaN,NaN,NaN,NaN,0.001399,0.641351
75%,61.905015,0.228011,19307.500000,265.370973,384.265263,NaN,NaN,NaN,NaN,0.002446,0.726746


Ce que nous remarquons dans un premier temps, c'est la présence des varibles *marge* et *warranty*, qui sont en plus par rapport à la description de l'énoncé. Cependant, la marge commerciale n'est autre que le prix moins le coût de production, aussi, il y a des chances que des **corrélations** aparaissent entre ces trois variable. C'est ce que nous vérifierons dans la phase suivante. 

Analysons maintenant en détail chaques variable de notre dataframe.

## 2. Analyse détaillée de chaque variable

### 2.1 La capacité

In [48]:
mower_df.capacity.describe()

count    1399.000000
mean       47.663266
std        19.840501
min         0.013884
25%        35.853890
50%        50.309242
75%        61.905015
max       104.614865
Name: capacity, dtype: float64

Des capacités comme la valeur min = 0.013 n'est pas possible puisque cette valeur est un litrage. Il est probable que certaines valeurs soient en m^^3^^ et non en litres (L). Pour en être certains, il faudrait s'en assurer auprès du métier. Dans notre cas, nous prendrons l'hypothèse que les capacités infiérieurs à 1 sont en m^^3^^, et nous les mettrons en L (* 1000).

### 2.2 Le taux de panne

In [49]:
mower_df.failure_rate.describe()

count    1399.000000
mean        0.166448
std         0.073605
min         0.028906
25%         0.095678
50%         0.199796
75%         0.228011
max         0.302465
Name: failure_rate, dtype: float64

Le taux de pannes sont entre 0 et 1, aucune incohérence majeure n'est à relever pour cette variable.

### 2.3. La marge

In [23]:
mower_df.margin.describe()

count    1399.000000
mean      186.262609
std       191.011102
min        52.943989
25%        62.247602
50%        66.299672
75%       265.370973
max       790.012898
Name: margin, dtype: float64

### 2.4. Le prix de vente

In [29]:
mower_df.price.describe()

count    1399.000000
mean      260.075165
std       259.949127
min        62.162200
25%        73.379425
50%        78.439753
75%       384.265263
max       964.973926
Name: price, dtype: float64

### 2.5. Le coût de fabrication

In [25]:
mower_df.prod_cost.describe()

count     1381
unique    1333
top          0
freq        20
Name: prod_cost, dtype: object

Cette colonne devrait être de type float64. Il y a donc potentiellement des Strings. Ceci explique que le "count" soit à 1381 et non à 1399: il doit y avoir des Strings vides. Il faudra donc effectuer une conversion pour analyser en profondeur cette variable.
 
### 2.6. Les types de produits

In [37]:
dfGroupBy = df.groupby("product_type")

In [40]:
dfGroupBy.describe()

capacity  failure_rate            id      margin  \
product_type                                                             
auto-portee  count  142.000000    142.000000    142.000000  142.000000   
             mean    18.199994      0.037475  19024.161972  683.913626   
             std      7.728494      0.003377    395.089821   39.139783   
             min      0.013884      0.028906  18268.000000  581.400879   
             25%     14.024788      0.035429  18726.500000  655.607890   
             50%     18.349754      0.037523  18998.500000  682.472046   
             75%     21.739153      0.039991  19375.500000  709.713343   
             max     39.566560      0.044026  19655.000000  790.012898   
electrique   count  413.000000    413.000000    413.000000  413.000000   
             mean    38.009129      0.095907  18958.624697  266.826354   
             std     12.084402      0.009290    397.203021   17.937542   
             min      0.022504      0.070670  18261.000000  220.341505   
             25%     32.824765      0.089401  18617.000000  255.741110   
             50%     38.415445      0.095586  18960.000000  265.863982   
             75%     45.198838      0.102027  19277.000000  277.854801   
             max     70.603826      0.124836  19654.000000  334.874460   
essence      count  844.000000    844.000000    844.000000  844.000000   
             mean    57.344479      0.222666  18946.562796   63.111814   
             std     16.903318      0.023162    408.152208    3.407393   
             min      0.043362      0.151225  18259.000000   52.943989   
             25%     50.892449      0.205492  18587.000000   60.663198   
             50%     58.814798      0.222732  18948.500000   63.081545   
             75%     67.181622      0.238305  19303.250000   65.387721   
             max    104.614865      0.302465  19657.000000   75.121280   

                         price  market_share  attractiveness  
product_type                                                  
auto-portee  count  142.000000    142.000000      142.000000  
             mean   821.349177      0.007042        0.720440  
             std    146.095136      0.000920        0.094094  
             min     69.359766      0.005066        0.518288  
             25%    819.375259      0.006314        0.645955  
             50%    846.982790      0.006979        0.713964  
             75%    882.497669      0.007653        0.782909  
             max    964.973926      0.009314        0.952877  
electrique   count  413.000000    413.000000      413.000000  
             mean   385.068490      0.002421        0.630903  
             std     86.231363      0.000500        0.130193  
             min     62.162200      0.001146        0.298501  
             25%    368.073290      0.002047        0.533243  
             50%    380.532676      0.002456        0.639941  
             75%    395.330818      0.002805        0.730958  
             max    895.908032      0.003519        0.916831  
essence      count  844.000000    844.000000      844.000000  
             mean   104.479012      0.001185        0.625127  
             std    141.064059      0.000228        0.120038  
             min     62.855737      0.000564        0.297384  
             25%     71.639465      0.001029        0.542652  
             50%     74.351156      0.001182        0.623783  
             75%     77.413470      0.001346        0.710374  
             max    910.237658      0.001736        0.915787

Les types ne correspondent pas à l'énoncé, on devrait avoir 1, 2, ou 3. La conversion sera possible en appliuant un dictionnaire sur cette variable.

### 2.7. L'indice de qualité

In [42]:
dfByQuality = df.groupby("quality")

In [43]:
dfByQuality.describe()

attractiveness    capacity  failure_rate            id  \
quality                                                                 
Hight   count      133.000000  133.000000    133.000000    133.000000   
        mean         0.644530   45.810637      0.156217  18978.849624   
        std          0.135740   20.463533      0.073417    353.926171   
        min          0.387732    0.015932      0.029659  18282.000000   
        25%          0.542442   33.926630      0.093689  18716.000000   
        50%          0.621892   49.682095      0.186253  18983.000000   
        75%          0.742652   59.795706      0.220487  19252.000000   
        max          0.952877   90.847604      0.261196  19647.000000   
Low     count      971.000000  971.000000    971.000000    971.000000   
        mean         0.630715   47.529465      0.167511  18962.617920   
        std          0.121469   19.701549      0.073630    402.423287   
        min          0.297384    0.013884      0.028906  18259.000000   
        25%          0.549390   35.820391      0.095949  18615.000000   
        50%          0.641172   50.138530      0.200876  18956.000000   
        75%          0.718845   61.479686      0.228515  19316.000000   
        max          0.894675  104.614865      0.302465  19657.000000   
Medium  count      295.000000  295.000000    295.000000    295.000000   
        mean         0.651955   48.938929      0.167560  18933.400000   
        std          0.125705   19.998311      0.073501    429.755803   
        min          0.348905    0.014320      0.031813  18260.000000   
        25%          0.554163   36.724379      0.095465  18510.000000   
        50%          0.644969   50.901964      0.201885  18954.000000   
        75%          0.742250   64.395756      0.230219  19301.000000   
        max          0.915787   83.650827      0.273140  19655.000000   

                   margin  market_share       price  
quality                                              
Hight   count  133.000000    133.000000  133.000000  
        mean   214.668704      0.002420  290.896620  
        std    210.630098      0.002302  270.623708  
        min     58.164394      0.000735   69.341855  
        25%     65.652572      0.001107   77.364718  
        50%     70.407105      0.001515   84.281920  
        75%    282.545552      0.002368  403.246334  
        max    790.012898      0.009314  935.661633  
Low     count  971.000000    971.000000  971.000000  
        mean   182.296985      0.002084  255.362849  
        std    186.475387      0.001641  257.008421  
        min     52.943989      0.000564   62.162200  
        25%     61.437378      0.001134   72.498218  
        50%     65.331966      0.001389   77.039972  
        75%    262.005867      0.002469  379.287178  
        max    756.350611      0.008053  945.613811  
Medium  count  295.000000    295.000000  295.000000  
        mean   186.508745      0.002219  261.690097  
        std    196.029443      0.001980  264.575821  
        min     56.669396      0.000661   65.758364  
        25%     64.136489      0.001142   75.608654  
        50%     67.376218      0.001406   79.773642  
        75%    268.494441      0.002356  389.629965  
        max    781.467462      0.008874  964.973926

La qualité est une variable catégorielle, pour plus de facilité, nous allons mapper les différentes catégories vers des entiers:

* Low : -1
* Medium : 0
* High : 1

### 2.8. La période de garantie

In [7]:
mower_df.warranty.values

array(['3ans', '3 ans.', '3_ans', ..., '1an.', '1 an', '1an.'], dtype=object)

Afin d'utiliser cette nouvelle variable, nous devrons l'uniformiser : 3ans, 3 ans. etc

### 2.9. Notre cible : l'attractivité

In [8]:
mower_df.attractiveness.describe()

count    1399.000000
mean        0.636507
std         0.124016
min         0.297384
25%         0.549369
50%         0.641351
75%         0.726746
max         0.952877
Name: attractiveness, dtype: float64

### 2.10 La part de marché

In [9]:
mower_df.market_share.describe()

count    1399.000000
mean        0.002144
std         0.001790
min         0.000564
25%         0.001135
50%         0.001399
75%         0.002446
max         0.009314
Name: market_share, dtype: float64

Comme spécifié dans l'énoncé, cette variable est une renormalisation de la variable "attractiveness", il y a donc un risque de **corr